In [1]:
import sys
import os

sys.path.append('..')
sys.path.append('.')

import pickle
import pandas as pd

from dice_integration import DICEVerifier
from property_verification import PropertyVerifier

F:\MU_LAB\Conda\envs\bn_c\lib\site-packages\pgmpy\utils\utils.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
F:\MU_LAB\Conda\envs\bn_c\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '../models/bn_trained_final2.pkl'

if not os.path.exists(model_path):
    print(f"Model not found: {model_path}")
    print(f"Current directory: {os.getcwd()}")
else:
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    
    print(f"Model loaded successfully")
    print(f"  Nodes: {len(model.nodes())}")
    print(f"  Edges: {len(model.edges())}")
    print(f"\nActual node names in model:")
    for i, node in enumerate(sorted(model.nodes()), 1):
        print(f"  {i}. {node}")

Model loaded successfully
  Nodes: 8
  Edges: 10

Actual node names in model:
  1. GSR
  2. HOG_features
  3. Left_Diameter
  4. Luminance
  5. Motion_Intensity
  6. Right_Diameter
  7. Temporal_Smoothness
  8. fms


In [3]:
try:
    verifier = DICEVerifier(
        bn_model_path='../models/bn_trained_final2.pkl',
        use_docker=True
    )
except Exception as e:
    print(f"Error initializing verifier: {e}")
    import traceback
    traceback.print_exc()

Loading model from: ../models/bn_trained_final2.pkl
Model loaded: 8 nodes
Nodes: ['GSR', 'HOG_features', 'Left_Diameter', 'Luminance', 'Motion_Intensity', 'Right_Diameter', 'Temporal_Smoothness', 'fms']


In [4]:
try:
    results = verifier.verify_with_dice(evidence=None)
    
    print("\nDICE Output:")
    print(results['output'])
    
    print("\nParsed Results:")
    if results['results']:
        for value, prob in sorted(results['results'].items()):
            print(f"  P(FMS = {value}) = {prob:.6f}")
        
        # Calculate P(FMS >= 4)
        p_severe = sum(prob for value, prob in results['results'].items() if value >= 4)
        print(f"\nP(FMS >= 4) = {p_severe:.6f}")
    else:
        print("  No results parsed (check DICE output above)")
        
except Exception as e:
    print(f"Verification failed: {e}")
    import traceback
    traceback.print_exc()

DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice

DICE Output:
================[ Joint Distribution ]================
Value	Probability	
0	0.2	
1	0.2	
2	0.15	
3	0.2	
4	0.15	
5	0.07	
6	0.03	
7	0.	



Parsed Results:
  P(FMS = 1) = 0.200000
  P(FMS = 2) = 0.200000
  P(FMS = 3) = 0.150000
  P(FMS = 4) = 0.200000
  P(FMS = 5) = 0.150000
  P(FMS = 6) = 0.070000
  P(FMS = 7) = 0.030000

P(FMS >= 4) = 0.450000


In [5]:
print("\n" + "="*60)
print("TEST 2: VERIFICATION WITH EVIDENCE")
print("="*60)

# Use CORRECT node names from the model
evidence = {'Motion_Intensity': 4, 'GSR': 5}
print(f"Evidence: {evidence}")

try:
    comparison = verifier.compare_with_pgmpy(evidence)
    
    if comparison is None:
        print("Comparison failed - invalid evidence")
    else:
        print("\nDICE Results:")
        for value, prob in sorted(comparison['dice'].items()):
            print(f"  P(FMS = {value} | evidence) = {prob:.6f}")
        
        print("\npgmpy Results:")
        for value, prob in sorted(comparison['pgmpy'].items()):
            print(f"  P(FMS = {value} | evidence) = {prob:.6f}")
        
        print("\nDifferences (|DICE - pgmpy|):")
        for value, diff in sorted(comparison['difference'].items()):
            print(f"  FMS = {value}: {diff:.6f}")
        
        print(f"\nMaximum Difference: {comparison['max_diff']:.6f}")
        
        if comparison['max_diff'] < 0.01:
            print("\n DICE and pgmpy agree (diff < 0.01)")
        else:
            print("\n Significant difference between DICE and pgmpy")
        
except Exception as e:
    print(f"Comparison failed: {e}")
    import traceback
    traceback.print_exc()



TEST 2: VERIFICATION WITH EVIDENCE
Evidence: {'Motion_Intensity': 4, 'GSR': 5}
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
Evidence: {'Motion_Intensity': 4, 'GSR': 5}

DICE Results:
  P(FMS = 1 | evidence) = 0.100000
  P(FMS = 2 | evidence) = 0.150000
  P(FMS = 3 | evidence) = 0.150000
  P(FMS = 4 | evidence) = 0.250000
  P(FMS = 5 | evidence) = 0.200000
  P(FMS = 6 | evidence) = 0.100000
  P(FMS = 7 | evidence) = 0.050000

pgmpy Results:
  P(FMS = 1 | evidence) = 0.122444
  P(FMS = 2 | evidence) = 0.069201
  P(FMS = 3 | evidence) = 0.061172
  P(FMS = 4 | evidence) = 0.077795
  P(FMS = 5 | evidence) = 0.326640
  P(FMS = 6 | evidence) = 0.167571
  P(FMS = 7 | evidence) = 0.175178

Differences (|DICE - pgmpy|):
  FMS = 1: 0.022444
  FMS = 2: 0.080799
  FMS = 3: 0.088828
  FMS = 4: 0.172205
  FMS = 5: 0.126640
  FMS = 6: 0.067571
  FMS = 7: 0.125178

Maximum Difference: 0.172205

 Si

In [6]:
print("\n" + "="*60)
print("TEST 3: FORMAL PROPERTY VERIFICATION")
print("="*60)

try:
    PropertyVerifier(model)
except Exception as e:
    print(f"Property verification failed: {e}")
    import traceback
    traceback.print_exc()


TEST 3: FORMAL PROPERTY VERIFICATION
PropertyVerifier initialized for model with 8 nodes


In [7]:
print("\n" + "="*60)
print("TEST 4: MULTIPLE SCENARIO ANALYSIS")
print("="*60)

scenarios = [
    {'name': 'Baseline', 'evidence': {}},
    {'name': 'High Motion', 'evidence': {'Motion_Intensity': 5}},
    {'name': 'High GSR', 'evidence': {'GSR': 5}},
    {'name': 'Both High', 'evidence': {'Motion_Intensity': 5, 'GSR': 5}},
    {'name': 'Both Low', 'evidence': {'Motion_Intensity': 1, 'GSR': 1}},
]

results_list = []

for scenario in scenarios:
    print(f"\nProcessing: {scenario['name']}")
    
    try:
        result = verifier.verify_with_dice(scenario['evidence'])
        
        p_severe = sum(
            prob for value, prob in result['results'].items() 
            if value >= 4
        )
        
        results_list.append({
            'Scenario': scenario['name'],
            'Evidence': str(scenario['evidence']),
            'P(FMS >= 4)': p_severe
        })
        
        print(f"   P(FMS >= 4) = {p_severe:.6f}")
        
    except Exception as e:
        print(f"  ✗ Failed: {str(e)[:50]}...")
        results_list.append({
            'Scenario': scenario['name'],
            'Evidence': str(scenario['evidence']),
            'P(FMS >= 4)': None
        })

# Display results table
print("\n" + "="*60)
print("SCENARIO ANALYSIS RESULTS")
print("="*60)

df = pd.DataFrame(results_list)
print(df.to_string(index=False))



TEST 4: MULTIPLE SCENARIO ANALYSIS

Processing: Baseline
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
   P(FMS >= 4) = 0.450000

Processing: High Motion
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
   P(FMS >= 4) = 0.600000

Processing: High GSR
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
   P(FMS >= 4) = 0.600000

Processing: Both High
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
   P(FMS >= 4) = 0.600000

Processing: Both Low
DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.d

In [8]:
results = verifier.verify_with_dice(evidence=None)

DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice


In [9]:
comparison = verifier.compare_with_pgmpy({'Motion_Intensity': 4, 'GSR': 5})

DICE program written to: C:\Users\Istiak Ahmed\Desktop\cybersickness_bn_wu2025\notebooks\dice_verification\verify.dice
Executing DICE on: verify.dice
Evidence: {'Motion_Intensity': 4, 'GSR': 5}
